<a href="https://colab.research.google.com/github/dalexa10/Python_Data_Science/blob/main/Materials_Analysis/CollectMaterialsProjectData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Gathering
Connect to database of Material Project and pull data to your computer



This notebook collects necessary data from Materials Project.

In [2]:
!pip3 install pymatgen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 98.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.1/515.1 KB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 KB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.3/555.3 KB 53.1 MB/s eta 0:00:0

In [4]:
import math
import multiprocessing
import json
import os.path
import ast
from pymatgen.core import periodic_table
# from pymatgen.core import Structure
# from pymatgen.matproj.rest import MPRester, MPRestError
from pymatgen.ext.matproj import MPRester, MPRestError
from pymatgen.analysis import bond_valence
import numpy as np
import matplotlib.pyplot as plt
import bisect
import re
import itertools
from scipy.signal import fftconvolve
from scipy import integrate
from scipy.constants import *
import copy

In the next cell enter your API key for Materials Project. Avoid logging each time and show your password with plain text 

In [5]:
API_Key=
mp=MPRester(API_Key)

SyntaxError: ignored

The following cell collects the used MPIDs in Materials project and writes to file.

In [6]:
IDFilename='idlist.txt'
fID=open(IDFilename,'w')

data = mp.query(criteria={}, properties=["task_id"])
AllIDs=[]
for i in data:
    AllIDs.append(int(i['task_id'].split('-')[1]))
IDList=list(set(AllIDs))
IDList.sort()
for i in IDList:
    fID.write(str(i)+'\n')
fID.close()
print('Done')

NameError: ignored

The next cell sets up the output files.

In [ ]:
filename2='CNData.txt'
g=open(filename2,'w')
g.write('#MP_Number'+'\t'+'CNL'+'\t'+'EG'+'\t'+'EFermi'+'\t'+'ValenceMax'+'\t'+'CondMin'+'\n')

filename3='OpticalProperties.txt'
h=open(filename3,'w')
h.write('#MP_NUmber'+'\t'+'Eps10'+'\t'+'Meff_El'+'\t'+'Meff_hole'+'\t'+'ExcitonEb'+'\t'+'EdgeJDOS'+'\n')

The next cell is collects.  This will take several hours.

In [ ]:
def opticalfunc(energies,gap,a,b): #this function approximates |optic|^2 using the gap
    optic=[b*abs((En-gap+delta))**(2*a) for En in energies]
    return optic

def ExcitonCalc(eps0,mH,mE): #Calculates Wannier Mott exciton binding energy
    return ERydberg*(mE*mH/(mE+mH))/eps0**2
    

############This is for separating into paths
def distance(a,b):
    dist=math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)          
    return dist

# This exception is an empty exception to handle no material at a given ID
class NoMaterialAtIDError(Exception):
    pass
    #Don't need anything here, just define it as an exception



def fractionalKpoints(ks):#dont worry about this name
    kMat=np.zeros(len(ks))
    oldDist=0.0
    recentSwitch='no'
    switchPosition=0
    for j in np.arange(1,len(ks)):
        dist=distance(ks[j-1],ks[j])
        if j==1:
            kMat[j]=dist
        elif j != 1 and abs(dist-oldDist)<.001 and recentSwitch=='no':#this is for same kpoint line
            kMat[j]=dist+kMat[j-1]
            recentSwitch='no'
        elif j !=1 and abs(dist-oldDist)>=.001 and recentSwitch=='no':#this is for switching kpoint lines
            kMat[j]=kMat[j-1]
            recentSwitch='yes'
            switchPosition=np.append(switchPosition,j)

        elif j!=1 and recentSwitch=='yes': #second point on new line
            kMat[j]=kMat[j-1]+dist
            recentSwitch='no'

        oldDist=dist
    kMat=kMat/max(kMat)
    return kMat,switchPosition

############from prior project

def EnergyGap(energies, VBtop, CBbottom): #give list of energies, index for highest valence and index for lowest conduction
    EGap=min(energies[CBbottom])-max(energies[VBtop])
    return EGap



def DatabaseChargeNeutrality(IDNum): #Charge Neutrality from materialsproject
    Name='mp-'+IDNum
    with MPRester() as mp:
        try:
            banddata = mp.get_data(Name, prop="bandstructure")
            if len(banddata) == 0:
                raise NoMaterialAtIDError       
            bandstruct = banddata[0]["bandstructure"]
            #name=mp.query(Name,["pretty_formula"])
            oxidizedstates = mp.query(Name, ["bv_structure"])[0]['bv_structure'].get_primitive_structure()
            total_e = 0
            #TODO: Investigate how specie_and_occu differs?
            #For now, assume we don't don't
            for specie in oxidizedstates.species:
                if specie.Z > MAX_ALLOWED_ELEMENT:
                    return
                try:
                    num_electrons = specie.Z+specie.oxi_state
                    #print(num_electrons)
                    
                    #skip if beyond known elements. This actually works for ions with
                    #no electrons (ie H^+ or He^2+)
                    if(num_electrons <= 0 or num_electrons > MAX_DEFINED_ELEMENT):
                        continue
                    #This is a hack of sorts. We get the element that has the same number 
                    #of electrons as our ion, and use that as the electronic structure
                    orbitals = periodic_table.get_el_sp(int(num_electrons)).full_electronic_structure
                    if len(orbitals) <= 1:
                        continue
                    index = -1
                    max_orb_num = orbitals[index][0]
                    #This will exclude d and higher electrons because they will never be the top
                    #principal quantum number. We could also grab this directly to be safe
                    while orbitals[index][0] == max_orb_num:
                        total_e += orbitals[index][2]
                        index -= 1
                except AttributeError:#this is for Si and possibly other single element compounds
                    total_e=8
            #print(total_e)
            NVal = total_e/4
            NCond = total_e/8
            #print(total_e)

            # Calculate branch point energy (ebp) and correct
            CNL,Eg,ValenceMax,ConductionMin=calcEBP(bandstruct, NVal, NCond, PRINT_DEBUG)#in here will also calculate optical properties
            FermiEn=bandstruct.efermi

        except NoMaterialAtIDError:
            if PRINT_DEBUG:
                print "Material " + Name + " does not exist!"
                return
            
        except MPRestError:
            if PRINT_DEBUG:
                print "No band struct or density of states for " + Name + "!"
                return
 
        #now print data if has gap w/correction
        if Eg>MetalThreshold:
            CNLs=str.format("{0:.3f}", CNL)
            Egs=str.format("{0:.3f}", Eg)
            FermiEns=str.format("{0:.3f}", FermiEn)
            ValenceMaxs=str.format("{0:.3f}", ValenceMax)
            ConductionMins=str.format("{0:.3f}", ConductionMin)
            #print(CNLs,Egs,FermiEns,ValenceMaxs,ConductionMins)
            g.write(IDNum+'\t'+CNLs+'\t'+Egs+'\t'+FermiEns+'\t'+ValenceMaxs+'\t'+ConductionMins+'\n')
        else:
            print "Material " + Name + " is a metal!"
        return


def calcEBP(bandstruct, N_VB, N_CB, PRINT_DEBUG): #this is for MatProj calculation

    bs = bandstruct
    NumBands=bs.nb_bands
    for i in range(bs.nb_bands):
        if (    np.mean(bs.bands[1][i+1]) > bs.efermi
            and abs(max(bs.bands[1][i+1])-max(bs.bands[1][i]))>0.001
            ): #check that i+1 band is on average above Ef and that the maxima of i+1 and i do not coincide
            cbbottom = i+1
            vbtop = i
            break #bands found first time condition is satisfied
    ConBanMin=min(bs.bands[1][cbbottom])
    ValBanMax=max(bs.bands[1][vbtop])
    
    for qq in range(bs.nb_bands):
        bs.bands[1][qq]=[x-bs.efermi for x in bs.bands[1][qq]]
    EVB=max(bs.bands[1][vbtop])
    for qq in range(bs.nb_bands):
        bs.bands[1][qq]=[x-EVB for x in bs.bands[1][qq]]
    
    #plt.plot(bs.bands[1][vbtop])
    #plt.plot(bs.bands[1][cbbottom])
    #plt.plot(bs.bands[1][vbtop-1])
    #plt.show()
    
    vb_en = 0.0 
    cb_en = 0.0
    
    kts=bandstruct.kpoints;
    ks=[];
        #Converts kpoints into fractional coordinates.
    for j in kts:
        ks.append(j.frac_coords);
    
    kMatP,switches=fractionalKpoints(ks)
    switches=np.append(switches,len(ks))#adds in ending point
    paths=np.zeros((len(switches)-1,4))
    pathWeights=np.zeros((len(switches)-1))
    for j in np.arange(0,len(switches)-1):
        paths[j,0]=switches[j]
        paths[j,1]=switches[j+1]-1
        paths[j,2]=paths[j,1]-paths[j,0]+1
        paths[j,3]=distance(ks[np.int(paths[j,0])],ks[np.int(paths[j,1])])
    totalLength=sum(paths[:,3])
    for j in np.arange(0,len(switches)-1): #weight only by path length
        pathWeights[j]=paths[j,3]/totalLength
    bandInfo=bs.bands[1]

    vb_PathEn=np.zeros(len(pathWeights))
    cb_PathEn=np.zeros(len(pathWeights))
    for i in range(0,len(pathWeights)):
        for j in range(N_VB):
            vb_PathEn[i]+=np.mean(bandInfo[vbtop-j][int(paths[i,0]):int(paths[i,1])+1])/N_VB*pathWeights[i]
        for j in range(N_CB):
            cb_PathEn[i]+=np.mean(bandInfo[cbbottom+j][int(paths[i,0]):int(paths[i,1])+1])/N_CB*pathWeights[i]
    
    ebp = (sum(vb_PathEn) + sum(cb_PathEn))/(2)
    Eg= EnergyGap(bs.bands[1],vbtop,cbbottom)#get energy gap

    if Eg>MetalThreshold: #to avoid calculating optical properties for metals
        epsilon0,edgeJDOS=CalcEps0(bandInfo, vbtop, cbbottom, NumBands, paths,Eg)
        #mElectron=CalcElectronMass(bandInfo, cbbottom, paths,ConBanMin-bs.efermi-EVB,ks)
        mHole=CalcHoleMass(bandInfo, vbtop, paths,ValBanMax-bs.efermi-EVB,copy.deepcopy(ks))
        mElectron=CalcElectronMass(bandInfo, cbbottom, paths,ConBanMin-bs.efermi-EVB,copy.deepcopy(ks))
        excitonEnergy=ExcitonCalc(epsilon0,mHole,mElectron)
        h.write(str(idElement)+'\t'+str(epsilon0)+'\t'+str(mElectron)+'\t'+str(mHole)+'\t'+str(excitonEnergy)+'\t'+str(edgeJDOS)+'\n')
    
    return ebp, Eg, ValBanMax, ConBanMin


def CalcEps0(struc,vtop,cbot,NBands,pathdat,gap): #calculates epsilon0
    valenceLabels=range(0,vtop+1)
    conductionLabels=range(cbot,NBands)
    bandcombinations=(list(itertools.product(valenceLabels,conductionLabels)))
    Energies=[] #v-c band energy differences
    Weights=[] #weight of k-point in jdos
    for ij in range(0,len(bandcombinations)): #constructs list of vertical transitions
        endiff=[a-b for a,b in zip(struc[bandcombinations[ij][1]],struc[bandcombinations[ij][0]])]
        Energies.extend(endiff)
    for ij in range(0,len(pathdat)): #list of weights for jdos integral
        pointweight=[1/(pathdat[-1][1]+1)]*pathdat[ij][2]#[pathdat[ij][3]/pathdat[ij][2]*JDOSScale]*pathdat[ij][2]
        Weights.extend(pointweight)
    Weights*=len(bandcombinations)
    shift=(slopeCorr-1)*gap+interceptCorr
    EnergiesShifted=[a+shift for a in Energies]
    hist,edges=np.histogram(EnergiesShifted,bins=1000,range=(0,20),weights=Weights)
    jdos=fftconvolve(hist, broadening, mode='same')
    newEdge=np.delete(edges,-1)
    newEdge[0]+=delta #move off 0
    
    opticalFit=opticalfunc(newEdge,gap+shift,ALPHA,BETA)
    JDOSIntegrand=[np.real(a*b/((c**2+delta*1j)*c)) for a,b,c in zip(jdos,opticalFit,newEdge)] #to be integrated for KK
    eps1jdos=1+32*math.pi*integrate.simps(JDOSIntegrand,newEdge)
    
    #Here find average of jdos near absorption edge
    jdosedge=[]
    for jdosloop in range(0,len(newEdge)):
        if gap+shift<=newEdge[jdosloop]<=gap+shift+absorptionRange:
            jdosedge.append(jdos[jdosloop])
    edgejdos=np.mean(jdosedge)
    #edgejdos=0.0
        
    return eps1jdos,edgejdos


def CalcElectronMass(bandInfo,cbbottom,paths,CondBandMin,kpoints):
    with MPRester() as mp:
        struct2=mp.get_structure_by_material_id('mp-'+str(idElement), True);
    latticeA=(struct2.lattice.a)*math.pow(10,-10)
    latticeB=(struct2.lattice.b)*math.pow(10,-10)
    latticeC=(struct2.lattice.c)*math.pow(10,-10)
    scale=[2*math.pi/latticeA,2*math.pi/latticeB,2*math.pi/latticeC]
    
    MEffectiveCond=[]
    for qq in range(len(paths)):
        Segment=bandInfo[cbbottom][int(paths[qq,0]):int(paths[qq,1])+1]
        SegmentMin=min(Segment)
        if (SegmentMin==CondBandMin and
            len(Segment)>2):#determine if band min is in the segment
            SegmentKPoints=kpoints[int(paths[qq,0]):int(paths[qq,1])+1]
            SegmentEnergies=bandInfo[cbbottom][int(paths[qq,0]):int(paths[qq,1])+1]
            MinIndex=Segment.index(min(Segment))
            #Above is the location of the 0 in this band
            #now find three k points and their energies for polynomial fit
            if MinIndex==0:
                fitKPoint1=SegmentKPoints[0] #this is the minimum
                fitKPoint2=SegmentKPoints[1]
                fitKPoint3=SegmentKPoints[2]
                fitEnergies=[SegmentEnergies[0],SegmentEnergies[1],SegmentEnergies[2]]
            elif MinIndex==len(Segment)-1:
                fitKPoint1=SegmentKPoints[MinIndex-2]
                fitKPoint2=SegmentKPoints[MinIndex-1]
                fitKPoint3=SegmentKPoints[MinIndex] #this is the minimum
                fitEnergies=[SegmentEnergies[MinIndex-2],SegmentEnergies[MinIndex-1],SegmentEnergies[MinIndex]]         
            else:
                fitKPoint1=SegmentKPoints[MinIndex-1]
                fitKPoint2=SegmentKPoints[MinIndex] #this is the minimum
                fitKPoint3=SegmentKPoints[MinIndex+1]
                fitEnergies=SegmentEnergies[MinIndex-1:MinIndex+2]
            
            for qqq in range(0,3):
                fitKPoint1[qqq]*=scale[qqq]
                fitKPoint2[qqq]*=scale[qqq]
                fitKPoint3[qqq]*=scale[qqq]
                fitEnergies[qqq]*=qscaled
            fitKDistances=distance(fitKPoint2,fitKPoint3)
            #fit=np.polyfit(fitKDistances,fitEnergies,2)
            if fitKDistances>0.0:
                fit=(fitEnergies[2]-2*fitEnergies[1]+fitEnergies[0])/(fitKDistances**2)
                if fit>0.0:
                    SegmentMass=(hbarSq/(fit))/m_e
                    MEffectiveCond.append(SegmentMass)
            #else:
            #    SegmentMass=10**6 #too flat -> large mass
    if MEffectiveCond:
        MEffCond=np.mean(MEffectiveCond)  
    else:
        MEffCond=10**4    
    return MEffCond


def CalcHoleMass(bandInfo,vbtop,paths,ValenceBandMax,kpoints):
    with MPRester() as mp:
        struct2=mp.get_structure_by_material_id('mp-'+str(idElement), True);
    latticeA=(struct2.lattice.a)*math.pow(10,-10)
    latticeB=(struct2.lattice.b)*math.pow(10,-10)
    latticeC=(struct2.lattice.c)*math.pow(10,-10)
    scale=[2*math.pi/latticeA,2*math.pi/latticeB,2*math.pi/latticeC]
    
    MEffectiveVal=[]
    for qq in range(len(paths)):
        Segment=bandInfo[vbtop][int(paths[qq,0]):int(paths[qq,1])+1]
        SegmentMax=max(Segment)
        if (SegmentMax==ValenceBandMax and
            len(Segment)>2):#determine if band min is in the segment
            SegmentKPoints=kpoints[int(paths[qq,0]):int(paths[qq,1])+1]
            SegmentEnergies=bandInfo[vbtop][int(paths[qq,0]):int(paths[qq,1])+1]
            MaxIndex=Segment.index(max(Segment))
            #Above is the location of the 0 in this band
            #now find three k points and their energies for polynomial fit
            if MaxIndex==0:
                fitKPoint1=SegmentKPoints[0] #this is the minimum
                fitKPoint2=SegmentKPoints[1]
                fitKPoint3=SegmentKPoints[2]
                fitEnergiesv=[SegmentEnergies[0],SegmentEnergies[1],SegmentEnergies[2]]
            elif MaxIndex==len(Segment)-1:
                fitKPoint1=SegmentKPoints[MaxIndex-2]
                fitKPoint2=SegmentKPoints[MaxIndex-1]
                fitKPoint3=SegmentKPoints[MaxIndex] #this is the minimum
                fitEnergiesv=[SegmentEnergies[MaxIndex-2],SegmentEnergies[MaxIndex-1],SegmentEnergies[MaxIndex]]         
            else:
                fitKPoint1=SegmentKPoints[MaxIndex-1]
                fitKPoint2=SegmentKPoints[MaxIndex] #this is the minimum
                fitKPoint3=SegmentKPoints[MaxIndex+1]
                fitEnergiesv=SegmentEnergies[MaxIndex-1:MaxIndex+2]
            for qqq in range(0,3):
                fitKPoint1[qqq]*=scale[qqq]
                fitKPoint2[qqq]*=scale[qqq]
                fitKPoint3[qqq]*=scale[qqq]
                fitEnergiesv[qqq]*=qscaled
            fitKDistances=distance(fitKPoint2,fitKPoint3)

            #fit=np.polyfit(fitKDistances,fitEnergies,2)
            if fitKDistances>0.0:
                fit=(fitEnergiesv[2]-2*fitEnergiesv[1]+fitEnergiesv[0])/(fitKDistances**2)
                if fit<0.0:
                    SegmentMass=(hbarSq/(fit))/m_e
                    MEffectiveVal.append(-1*SegmentMass)
            #else:
            #    SegmentMass=-10**6 #too flat -> large negative mass
    if MEffectiveVal:
        MEffVal=np.mean(MEffectiveVal)
    else:
        MEffVal=10**4  
    
    return MEffVal


#############self written

######################
# ----------------- FIND ACTIVE LAYER/BLOCKING LAYER MATERIALS ----------------
# ------- Debug/Output Settings -------

PRINT_DEBUG = True
PRINT_INFO = True
SAVE_INFO = True

# ------- Global settings for linear correction -------
EGAP_CORRECTION_SLOPE = 1.242
EGAP_CORRECTION_Y_INT = .975


MAX_DEFINED_ELEMENT = 200
MAX_ALLOWED_ELEMENT = 200
#####################
hbarSq=hbar**2
qscaled=e
ERydberg=13.60569
###################
hfont = {'fontname':'Times New Roman'}
MetalThreshold=-1.0 #threshold gap to be considered a metal and filtered out
delta=10**-6 #for moving poles off real energy axis
slopeCorr=1.348
interceptCorr=0.913
xcoords=np.linspace(-2, 2, 200)
sigma=0.1
broadening=[1/(math.sqrt(2*math.pi*sigma**2))*math.exp((-a**2)/(2*sigma**2)) for a in xcoords]

ALPHA=-0.0575
BETA=0.568

absorptionRange=0.5 #range of energies over which to average jdos near gap edge

data = mp.query(criteria={}, properties=["task_id"]) #list of used MPIDs
MPIDList=[]
for i in data:
    id=i['task_id']
    MPIDList.append(int(re.sub("[^0-9]", "",str(id))))
MPIDList=list(set(MPIDList)) #remove duplicate values in MPIDList????
MPIDList.sort(key=int) #sorted list of used MPIDs

IDs=np.genfromtxt('idlist.txt',dtype=int)
IDs=[int(a) for a in IDs]

#now get data from MP
for idElement in IDs:#MPIDList:
    if idElement>=0:
        print idElement
        try:
            DatabaseChargeNeutrality(str(int(idElement)))
        except:
            pass

g.close()
h.close()
print('Done')

The next cell collects descriptors for the desired materials.

In [ ]:
MaterialElectronicData=np.loadtxt('CNData.txt')
filename3='Descriptors22.txt' #This holds physical data
filename4='ChemComps22.txt' #This holds chemical formulas 851031


def findDirect(bs): #will determine if band structure is direct or indirect
    for i in range(bs.nb_bands):
        if (    np.mean(bs.bands[1][i+1]) > bs.efermi
            and abs(max(bs.bands[1][i+1])-max(bs.bands[1][i]))>0.001
            ):
            cbbottom = i+1
            vbtop = i
            break
    '''plt.plot(bs.bands[1][vbtop])
    plt.plot(bs.bands[1][cbbottom])
    plt.plot(bs.bands[1][vbtop-1])
    plt.show()'''
    
    CBM = min(bs.bands[1][cbbottom])
    CMins= [i for i, v in enumerate(bs.bands[1][cbbottom]) if v == CBM]
    VBM= max(bs.bands[1][vbtop])
    VMaxes= [i for i, v in enumerate(bs.bands[1][vbtop]) if v == VBM]
    different= set(CMins).isdisjoint(VMaxes) #True means no shared elements
    if different: #in case of smearing
        isDirect=False #no shared elements->not direct
    else:
        isDirect=True
    return isDirect
class NoMaterialAtIDError(Exception):
    pass
    #Don't need anything here, just define it as an exception


h=open(filename3,'w')
k=open(filename4,'w')
h.write('#MP_Number'+'\t'+'Density'+'\t'+'EHull'+'\t'
        +'Volume'+'\t'+'Symm'+'\t'+'formE'+'\t'+'a'+'\t'+'b'+'\t'+'c'+'\t'+'NumAtoms'+'\t'+'Direct'+'\n')
k.write('#MP_Number'+'Chemical_Formula'+'\n')
MaybeCount=0
for i in range(0,len(MaterialElectronicData)): #here will get densities from materials project
    MPName='mp-'+str(int(MaterialElectronicData[i][0]))
    with MPRester() as mp: 
        Data=MaterialElectronicData[i]
        data = mp.query(criteria={"task_id": MPName}, properties=["pretty_formula",'volume','spacegroup.number','formation_energy_per_atom',
                                    'structure','density','e_above_hull','nsites'])
        ID=str(int(MaterialElectronicData[i][0]))
        composition=str(data[0]['pretty_formula'])
        volume=str.format("{0:.6f}",data[0]['volume'])
        symmGroup=str(data[0]['spacegroup.number'])
        formE=str.format("{0:.6f}",data[0]['formation_energy_per_atom'])
        lattA=str.format("{0:.6f}",data[0]['structure'].lattice.a)
        lattB=str.format("{0:.6f}",data[0]['structure'].lattice.b)
        lattC=str.format("{0:.6f}",data[0]['structure'].lattice.c)
        density=str.format("{0:.6f}",data[0]['density'])
        eHull=str.format("{0:.6f}",data[0]['e_above_hull'])
        NAtom=str(int(data[0]['nsites']))
        #Now determine if gap is direct or indirect
        try:
            banddata= mp.get_data(MPName, prop="bandstructure")
            bandstruct = banddata[0]["bandstructure"]
            isdirect=findDirect(bandstruct) #determines if direct or indirect bandstructure
        except NoMaterialAtIDError:
            isdirect='Maybe'
            MaybeCount+=1
        except MPRestError:
            isdirect='Maybe'
            MaybeCount+=1
        for item in [ID,density,eHull,volume,symmGroup,formE,lattA,lattB,lattC,NAtom,isdirect]:
            h.write("%s\t" % item)
        for item in [ID,composition]:
            k.write("%s\t" % item)
        h.write('\n')
        k.write('\n')
        #print (int(MaterialElectronicData[i][0]),volume,density, symmGroup,formE,lattA,lattB,lattC)
        #h.write(ID+'\t'+str(MaterialElectronicData[i][1])+'\t'+str(MaterialElectronicData[i][2])+'\t'+str(MaterialElectronicData[i][3])+'\t'+str(MaterialElectronicData[i][4])+'\t'+volume+'\t'+symmGroup+'\t'+formE+'\t'+'\t'+lattA+'\t'+lattB+'\t'+lattC+'\n')
h.close()
k.close()

print('Done')